In [8]:
# import numpy as np
# import tensorflow as tf
# # import random as rn

# import keras
# from keras import backend as K
# from keras.models import Sequential
# from keras.layers import Activation, Dropout
# from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
# from keras.metrics import categorical_crossentropy
# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import *
# from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
# import itertools
# import matplotlib.pyplot as plt
# %matplotlib inline

# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [9]:
filepath_files = '/Users/krahman/work/tutorials/tensorflow_classification/data/'

df_features = pd.read_csv(filepath_files + 'df_imputed.csv').drop('Unnamed: 0', axis=1)
df_features = shuffle(df_features).reset_index(drop=True)

train_samples, test_samples = train_test_split(df_features, test_size=.2)
train_samples, val_samples = train_test_split(train_samples, test_size=.2)

train_labels = np.array(train_samples.pop('isFraud'))
test_labels = np.array(test_samples.pop('isFraud'))
val_labels = np.array(val_samples.pop('isFraud'))

print("Training data shape:", train_samples.shape)
print("Validation data shape:", val_samples.shape)
print("Testing data shape:", test_samples.shape)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
val_samples = np.array(val_samples)

# scaler = MinMaxScaler(feature_range=(0,1))
scaler = StandardScaler()
scaled_train_samples = scaler.fit_transform(train_samples)
scaled_test_samples = scaler.transform(test_samples)
scaled_val_samples = scaler.transform(val_samples)

loss = keras.losses.BinaryCrossentropy()

Training data shape: (377945, 227)
Validation data shape: (94487, 227)
Testing data shape: (118108, 227)


In [11]:
# testing softmax
neg, pos = np.bincount(df_features.isFraud)
initial_bias = np.log([pos/neg])
output_bias = tf.keras.initializers.Constant(initial_bias)

    
# print(output_bias)
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

epochs = 2
lr = .0001

model = keras.Sequential([keras.layers.Dense(64, activation='relu', input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(64, activation='relu'),
#                           keras.layers.Dropout(0.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# base model predictions
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
print('Base model:')
print(cm)

model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=100, epochs=epochs, shuffle=True, verbose=2, workers=4, 
          use_multiprocessing=True)

predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
cm = confusion_matrix(test_labels, rounded_predictions)
print(cm)

Base model:
[[113789    107]
 [  4194     18]]
Train on 377945 samples, validate on 94487 samples
Epoch 1/2
377945/377945 - 25s - loss: 0.1188 - auc: 0.8175 - fp: 519.0000 - fn: 11385.0000 - val_loss: 0.1116 - val_auc: 0.8434 - val_fp: 111.0000 - val_fn: 2760.0000
Epoch 2/2
377945/377945 - 21s - loss: 0.1099 - auc: 0.8528 - fp: 624.0000 - fn: 10737.0000 - val_loss: 0.1072 - val_auc: 0.8555 - val_fp: 170.0000 - val_fn: 2599.0000
[[113702    194]
 [  3395    817]]
